In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
dataset_params = {
    'data_path': 'timeserie/timeserie_000.npy',
    'lambda': 1.25,
    'alpha': 1.05,
    'step_size': 0.07,
    'min_stability_duration': 1,
    'window_size': 64,
    'train_size': 0.75,
    'horizon': 1,
    'feature_type': 'stationary',
    'balancing': 'undersampling',
    'pos_ratio': 0.5,
    'lag': 1,
}
MODEL = 'MLP'
# lstm_model_params = {
#     'lstm': {
#         'input_dim': 1,
#         'hidden_dim': 512,
#         'num_layers': 3,
#         'dropout': 0.1,
#     },
#     'linear': {}
# }
mlp_model_params = {
    'hidden_dim': 256,
    'n_hidden': 3,
}
experiment_params = {
    'lr': 1e-6,
    'batch_size': 8,
    'num_epochs': 20
}

In [3]:
import numpy as np
from cpd_nn.dataset import SynthDatasetClassification
from cpd_nn.model import CPD_MLP
from cpd_nn.train import train
import wandb
import torch

with open(dataset_params['data_path'], 'rb') as f:
        ts = np.load(f)
ts = ts[:10000]
sd = SynthDatasetClassification(
    ts, dataset_params['min_stability_duration'], 
    dataset_params['window_size'],
    dataset_params['train_size'], 
    dataset_params['horizon'],
    dataset_params['feature_type'],
    dataset_params['balancing'],
    dataset_params['pos_ratio'],
    lap=dataset_params['lag']
)

In [5]:
train_loader = sd.get_trainloader(  
    batch_size=experiment_params['batch_size'],
    shuffle_train=True,
    model_type=MODEL
)
val_loader = sd.get_trainloader(  
    batch_size=experiment_params['batch_size'],
    model_type=MODEL
)

In [6]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [7]:
model = CPD_MLP(input_dim=dataset_params['window_size'], hidden_dim=mlp_model_params['hidden_dim']).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=experiment_params['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

In [8]:
metrics = train(model, optimizer, loss_fn, train_loader, val_loader, experiment_params['num_epochs'], DEVICE, use_wandb=False, calc_metric=False)

Epoch 11 [Train]:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 11 [Val]:   0%|          | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 